## Load the list of URL's to get the food hygene data

In [1]:
from bs4 import BeautifulSoup
import requests

url = "https://ratings.food.gov.uk/open-data/en-GB"
xml_data = requests.get(url).content 
URLs= []


soup = BeautifulSoup(xml_data)

links = soup.find_all("td")
for link in links:
    URL = link.find('a', href=True)
    if URL is None:
        continue    
    rows = URL['href']
    URLs.append(rows)        



## Now process those URL's to get the food hygene data into a datafile

In [2]:
headers = {'accept': 'application/xml;q=0.9, */*;q=0.8'}
data = []
data.append (["Business Name", "Business Type", "PostCode", "RatingValue", "RatingDate", "LocalAuthorityName", "LocalAuthorityCode","Geocode_Lat", "Geocode_Long", "Hygiene", "Structural", "Confidence In Management"])

#URLs = []
#URLs.append("https://ratings.food.gov.uk/OpenDataFiles/FHRS858en-GB.xml")

print ("Number of URLs to process:", len(URLs))
counter = 0

for URL in URLs:
    link = URL

    # counter just to give us an idea how far through we are
    counter +=1

    # use this counter when testing to restrict amount of data loaded
    if counter > 2000:
        break

    print (str(counter), "/", str(len(URLs)), link)

    XMLdata = requests.get(link, headers=headers)

    # Passing the stored data inside the beautifulsoup parser 
    soup = BeautifulSoup(XMLdata.text, features = "xml") 


    Establishments = soup.find("EstablishmentDetail")
     

    #Extact all the next siblings of the element

    while Establishments != None:
        BusinessName = Establishments.find("BusinessName")
        BusinessType = Establishments.find("BusinessType")
        PostCode = Establishments.find("PostCode") 
        RatingValue = Establishments.find("RatingValue")
        RatingDate = Establishments.find("RatingDate")
        LocalAuthorityName = Establishments.find("LocalAuthorityName")
        LocalAuthorityCode = Establishments.find("LocalAuthorityCode")

        PostCode = Establishments.find("PostCode")

        if PostCode:
            vPostCode = PostCode.text
        else:
            vPostCode = '-'

        GeoCode = Establishments.find("Geocode") 
        Geocode_Longtitude = GeoCode.find("Longitude")
        Geocode_Latitude = GeoCode.find("Latitude")
       
        if Geocode_Longtitude == None:
            Geocode_Long = "none"
        else:
                Geocode_Long = Geocode_Longtitude.text
        if Geocode_Latitude == None:
            Geocode_Lat = "none"
        else:
            Geocode_Lat = Geocode_Latitude.text
    
        Scores = Establishments.find("Scores") 
        Scores_Hygiene = Scores.find("Hygiene")
        Scores_Structural = Scores.find("Structural")
        Scores_ConfidenceInManagement = Scores.find("ConfidenceInManagement")
       
        if Scores_Hygiene == None:
            Hygiene = "none"
        else:
            Hygiene = Scores_Hygiene.text
        if Scores_Structural == None:
            Structural = "none"
        else:
                Structural = Scores_Structural.text
        if Scores_ConfidenceInManagement == None:
            ConfidenceInManagement = "none"
        else:
            ConfidenceInManagement = Scores_ConfidenceInManagement.text
        
        
        Establishments = Establishments.find_next_sibling()
        
        # only save the data if we have a lat/long value to avoid mapping errors
        
        if Geocode_Lat != 'none' or Geocode_Long != 'none':
            rows = [BusinessName.text,BusinessType.text, vPostCode, RatingValue.text, RatingDate.text, LocalAuthorityName.text, LocalAuthorityCode.text, Geocode_Lat, Geocode_Long, Hygiene, Structural, ConfidenceInManagement]
            data.append(rows)


Number of URLs to process: 398
1 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS297en-GB.xml
2 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS109en-GB.xml
3 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS701en-GB.xml
4 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS110en-GB.xml
5 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS227en-GB.xml
6 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS111en-GB.xml
7 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS228en-GB.xml
8 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS155en-GB.xml
9 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS027en-GB.xml
10 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS112en-GB.xml
11 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS702en-GB.xml
12 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS113en-GB.xml
13 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS114en-GB.xml
14 / 398 https://ratings.food.gov.uk/OpenDataFiles/FHRS156en-GB.xml
15 / 398 https://ratings.f

## Save the data as CSV

In [6]:
import csv

with open('FoodRating.csv', 'w', newline = '', encoding ="utf-8") as f:
    csv_writer= csv.writer(f)
    csv_writer.writerows (data)

## Clean up our data a bit by removing the stuff we can't easily use

In [7]:
import pandas as pd

df = pd.read_csv("FoodRating.csv")

# Only select records with rathing 0-5

df1 = df[(df.RatingValue).isin(["0", "1", "2", "3", "4", "5"])]
df2 = df1[(df.PostCode) != "-"]



C:\Users\nigel\AppData\Local\Temp\ipykernel_394836\2262874864.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df1[(df.PostCode) != "-"]


## Categorise the business type


In [8]:

for index, row in df2.iterrows():
    if row['Business Type'] == "Takeaway/sandwich shop":
        value = 1
    elif row['Business Type'] == "Retailers - other":
        value = 2
    elif row['Business Type'] == "Pub/bar/nightclub":
        value = 3
    elif row['Business Type'] == "Restaurant/Cafe/Canteen":
        value = 4
    elif row['Business Type'] == "Pub/bar/nightclub":
        value = 5
    elif row['Business Type'] == "Other catering premises":
        value = 6
    elif row['Business Type'] == "Retailers - supermarkets/hypermarkets":
        value = 7
    elif row['Business Type'] == "Hospitals/Childcare/Caring Premises":
        value = 8
    elif row['Business Type'] == "School/college/university":
        value = 9
    elif row['Business Type'] == "Mobile caterer":
        value = 10
    elif row['Business Type'] == "Distributors/Transporters":
        value = 11
    elif row['Business Type'] == "Hotel/bed & breakfast/guest house":
        value = 12  
    elif row['Business Type'] == "Manufacturers/packers":
        value = 13
    elif row['Business Type'] == "Importers/Exporters":
        value = 14
    elif row['Business Type'] == "Farmers/growers":
        value = 15
    else:
        value = 100

    df2.loc[index, 'Business Type Index'] = value

    ## process the postcode to remove the spaces due to inconsistencies matching with post code file

    PostCode_S = row['PostCode'].replace(" ","")
    df2.loc[index, 'PostCode_S'] = PostCode_S
      


C:\Users\nigel\AppData\Local\Temp\ipykernel_394836\951342922.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[index, 'Business Type Index'] = value
C:\Users\nigel\AppData\Local\Temp\ipykernel_394836\951342922.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[index, 'PostCode_S'] = PostCode_S


## Save our file

In [9]:
data = df2.to_csv("FoodRating.csv", index=False)